In [37]:
import pandas as pd
import numpy as np
import sys
import random

from PIL import Image
from os import listdir
from os.path import isfile, join
from sklearn.neural_network import MLPClassifier

In [38]:
main_path = 'C:/Users/Rik/Documents/Radboud Universiteit/TFfCA/Data/CFD Version 2.0.3/'
excel_path = main_path + 'CFD 2.0.3 Norming Data and Codebook.xlsx'
pictures_path = main_path + 'CFD 2.0.3 Images/'
cfd = pd.read_excel(excel_path, sheet_name = 0, header = 4, skipRows = 4)
cfd.head()

,Target,Race,Gender,Age,NumberofRaters,Female_prop,Male_prop,Asian_prop,Black_prop,Latino_prop,...,EyeShape,EyeSize,UpperHeadLength,MidfaceLength,ChinLength,ForeheadHeight,CheekboneHeight,CheekboneProminence,FaceRoundness,fWHR
0,AF-200,A,F,32.571429,28,1.000000,0.000000,1.000000,0.000000,0.000000,...,0.404651,0.060924,0.414099,0.326797,0.130719,0.264706,0.388189,91.5,0.545752,1.921146
1,AF-201,A,F,23.666667,27,1.000000,0.000000,0.962963,0.000000,0.000000,...,0.329204,0.041892,0.414414,0.329279,0.144595,0.300901,0.383784,146.0,0.488288,1.901129
2,AF-202,A,F,24.448276,29,0.827586,0.172414,0.310345,0.068966,0.137931,...,0.357939,0.051586,0.411080,0.310317,0.173424,0.298475,0.397029,58.0,0.481333,1.888249
3,AF-203,A,F,22.758621,29,1.000000,0.000000,0.758621,0.000000,0.068966,...,0.443200,0.063913,0.354407,0.343793,0.169820,0.272266,0.421089,87.5,0.500231,1.863719
4,AF-204,A,F,30.137931,29,1.000000,0.000000,0.827586,0.000000,0.068966,...,0.432247,0.053435,0.438931,0.293045,0.180237,0.293893,0.371925,73.5,0.513571,1.935783


In [43]:
def encodeRace(race):
    #Asian = 1, Black = 2, Latin = 3, White = 4, Other = 0
    if(race is 'A'):
        return 1
    elif(race is 'B'):
        return 2
    elif(race is 'L'):
        return 3
    elif(race is 'W'):
        return 4
    else:
        return 0

In [44]:
def encodeGender(gender):
    #Female = 1, Male = 2
    if(gender is 'F'):
        return 1
    elif(gender is 'M'):
        return 2
    else:
        return 0

In [39]:
cfdOrdinal = cfd.copy(deep=True)
cfdOrdinal['Race'] = cfdOrdinal['Race'].apply(lambda x: encodeRace(x))
cfdOrdinal['Gender'] = cfdOrdinal['Gender'].apply(lambda y: encodeGender(y))
cfdOrdinal = cfdOrdinal.drop('Suitability', axis = 1)
cfdOrdinal = cfdOrdinal.drop('NumberofRaters', axis = 1)
cfdOrdinal.head()

,Target,Race,Gender,Age,Female_prop,Male_prop,Asian_prop,Black_prop,Latino_prop,Multi_prop,...,EyeShape,EyeSize,UpperHeadLength,MidfaceLength,ChinLength,ForeheadHeight,CheekboneHeight,CheekboneProminence,FaceRoundness,fWHR
0,AF-200,1,1,32.571429,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.404651,0.060924,0.414099,0.326797,0.130719,0.264706,0.388189,91.5,0.545752,1.921146
1,AF-201,1,1,23.666667,1.000000,0.000000,0.962963,0.000000,0.000000,0.037037,...,0.329204,0.041892,0.414414,0.329279,0.144595,0.300901,0.383784,146.0,0.488288,1.901129
2,AF-202,1,1,24.448276,0.827586,0.172414,0.310345,0.068966,0.137931,0.448276,...,0.357939,0.051586,0.411080,0.310317,0.173424,0.298475,0.397029,58.0,0.481333,1.888249
3,AF-203,1,1,22.758621,1.000000,0.000000,0.758621,0.000000,0.068966,0.172414,...,0.443200,0.063913,0.354407,0.343793,0.169820,0.272266,0.421089,87.5,0.500231,1.863719
4,AF-204,1,1,30.137931,1.000000,0.000000,0.827586,0.000000,0.068966,0.103448,...,0.432247,0.053435,0.438931,0.293045,0.180237,0.293893,0.371925,73.5,0.513571,1.935783


In [77]:
def accessPicture(target):
    testPath = pictures_path + "" + target
    allFilesInPath = [f for f in listdir(testPath) if isfile(join(testPath, f))]
    neutralPic = next((i_path for i_path in allFilesInPath if 'N' in i_path), (""))
    return Image.open(testPath + '/' + neutralPic)

In [78]:
testtup = ("NF-123", True)
print(testtup[0])

NF-123


In [80]:
#Takes an array of picture names as input, returns an array of all pixel values
def buildMLPtestInput(pictureNames):
    MLPinput = []
    for i in range(0, len(pictureNames)):
        subject = list(cfdOrdinal.loc[cfdOrdinal["Target"] == pictureNames[i]].values[0])
        del subject[0]
        MLPinput.append(subject)
        print('{} / {} complete.'.format(i+1,len(pictureNames)), end='\r')
        sys.stdout.flush()
    return MLPinput

In [81]:
def createRandomTrainSet(trainSet):
    networkIn = []
    for i in range(0, len(trainSet)):
        networkIn.append((trainSet[i],random.choice([True, False])))
    return networkIn

In [109]:
#Takes an array of picture names as input, returns an array of all pixel values
def buildMLPtrainInput(pictureTuples):
    MLPinput = []
    MLPlabels = []
    for i in range(0, len(pictureTuples)):
        current = pictureTuples[i]
        subject = list(cfdOrdinal.loc[cfdOrdinal["Target"] == current[0]].values[0])
        del subject[0]
        MLPinput.append(subject)
        MLPlabels.append(current[1])
        print('{} / {} complete.'.format(i+1,len(pictureTuples)), end='\r')
        sys.stdout.flush()
    return MLPinput, MLPlabels

In [110]:
#Random initialization for testing purposes
#This part creates a training set
exampleTrainSet = cfd.head(20)
etsNames = exampleTrainSet["Target"].tolist()
exampleInput = createRandomTrainSet(etsNames)
mlpInput = buildMLPtrainInput(exampleInput)

In [111]:
#Random initialization for testing purposes
#This part creates a small test set
exampleTestSet = cfd.loc[5]
# print(exampleTestSet["Target"])
testInput = buildMLPtestInput([exampleTestSet["Target"]])

In [112]:
clf = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(1, 34), random_state=1, verbose = True)

In [113]:
clf.fit(mlpInput[0], mlpInput[1])

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1, 34), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=True,
       warm_start=False)

In [115]:
MLPoutput = clf.predict(testInput)
MLPoutput[0]

False

In [9]:
#Should result in array of 12,596,376 values
def imageToFloats(image):
    red = []
    green = []
    blue = []
    for i in range(0, image.size[0]):
        for j in range(0, image.size[1]):
            r,g,b = image.getpixel((i,j))
            red.append(r)
            green.append(g)
            blue.append(b)
    return red + green + blue

In [42]:
#Remove the 'Target' column prior to calling this function
def checkMat():
    for row in range (0, len(checkArray)):
        for col in range (0, len(checkArray[0])):
            if(type(checkArray[row][col]) is not np.float64):
                return False
    return True
# checkArray = cfdOrdinal.as_matrix()
# checkMat()

In [30]:
# races = cfd.Race.unique()
# genders = cfd.Gender.unique()
# print("Races in dataset:")
# for race in races:
#     print(race, ": ", len(cfd.loc[cfd['Race'] == race]))
# print('='*50)

# print("Genders in dataset:")
# for gender in genders:
#     print(gender, ": ", len(cfd.loc[cfd['Gender'] == gender]))
# print('='*50)

Races in dataset:
A :  109
B :  197
L :  108
W :  183
Genders in dataset:
F :  307
M :  290
